In [18]:
import numpy as np
import pandas as pd
import email
from html.parser import HTMLParser
from bs4 import BeautifulSoup
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from email.parser import BytesParser
from email.policy import default
import csv
import os
import pathlib
import pprint
from os.path import dirname, abspath

In [29]:
d = dirname(dirname(abspath(__file__)))

In [38]:
location = f"{d}/case study 3/"

In [44]:
emails = {"index":[]}
for path, subdirectories, filelist in os.walk(os.path.join(location,"SpamAssassinMessages")):
    for f in filelist:
        
        label = 0 if "ham" in path.lower() else 1
        with open(os.path.join(path, f), "rb") as fp:
            msg = BytesParser(policy=default).parse(fp)
            with open(os.path.join(path, f), "r",encoding= 'utf-8', errors='ignore') as new_f:
                msg = email.message_from_file(new_f)
                for part in msg.walk():
                    if part.get_content_type() == 'text/plain':
                        body = part.get_payload()  # decode
                emails["index"].append({
                    "Subject": msg.get("Subject"),
                    "From": msg.get("from"),
                    "Body": body,
                    "isSpam": label,
                    "FileID": f
                })

df = pd.DataFrame.from_dict(emails["index"])

In [45]:
df["isSpam"].value_counts()

0    6954
1    2400
Name: isSpam, dtype: int64

In [29]:
pd.set_option('display.max_rows', 9000)
pd.set_option('display.max_columns', 200)

Build a spam classifier using naive Bayes and clustering. You will have to create your own dataset from the input messages. Be sure to document how you created your dataset.

In [28]:
df["isSpam"].value_counts()

0    6954
1    2400
Name: isSpam, dtype: int64

In [6]:
df.isna().sum()

Subject    25
From        6
Body        0
isSpam      0
FileID      0
dtype: int64

In [31]:
df.head(9000)

In [12]:
df = df.dropna(how='any',axis=0) 

In [13]:
df.isna().sum()

Subject    0
From       0
Body       0
isSpam     0
dtype: int64

In [15]:
df["isSpam"].value_counts()

0    6944
1    2383
Name: isSpam, dtype: int64

In [16]:
#remove new lines and such
df = df.replace(r'\\n',' ', regex=True)
df = df.replace(r'\\t',' ', regex=True)
df = df.replace(r'b\'',' ', regex=True)
df = df.replace(r'b\"',' ', regex=True)


,Subject,From,Body,isSpam
9349,RE: [Razor-users] What's wrong with the Razor ...,"""Rose, Bobby"" <brose@med.wayne.edu>","I'm one of the 30,000 but it's not working ve...",0
9350,Re: W3C approves HTML 4 'emotitags' [...],harley@argote.ch,Damien Morton quoted: >W3C approves HTML 4 'e...,0
9351,Re: Ximian apt repos?,Mark Derricutt <mark@talios.com>,"On Mon, 2002-07-22 at 06:50, che wrote: > th...",0
9352,Re: Installing RPM,Matthias Saou <matthias@egwn.net>,"Once upon a time, Manfred wrote : > I would ...",0
9353,"scan bug: no update after Pick's ""New FTOC""",Brent Welch <welch@panasas.com>,"If you run Pick, and then use the ""New FTOC"" ...",0
